In [8]:
import os, time, json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
from pathlib import Path


In [9]:
# CONFIG GLOBALE
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42

EPOCHS_HEAD = 10
EPOCHS_FINE = 15

LR_HEAD = 1e-3
LR_FINE = 1e-4

DATASET_DIR = Path("/home/admin1/PROJETS_KEYCE/cc_transfert/dataset")
TRAIN_DIR = DATASET_DIR / "train"
VAL_DIR   = DATASET_DIR / "val"

assert TRAIN_DIR.exists() and VAL_DIR.exists()
print("✅ Dataset OK")


✅ Dataset OK


**Data Augmentation**

In [10]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.05,
    height_shift_range=0.05,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True
)
val_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [11]:
train_ds = train_gen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True,
    seed=SEED
)
val_ds = val_gen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)
NUM_CLASSES = train_ds.num_classes
CLASS_NAMES = list(train_ds.class_indices.keys())

print("Classes:", CLASS_NAMES)
print("NUM_CLASSES:", NUM_CLASSES)

Found 1840 images belonging to 10 classes.


Found 466 images belonging to 10 classes.
Classes: ['crease', 'crescent_gap', 'inclusion', 'oil_spot', 'punching_hole', 'rolled_pit', 'silk_spot', 'waist_folding', 'water_spot', 'welding_line']
NUM_CLASSES: 10


**Modèle ResNet50**

In [12]:
def build_resnet50(num_classes, lr):
    base_model = ResNet50(
        include_top=False,
        weights="imagenet",
        input_shape=(*IMG_SIZE, 3)
    )
    base_model.trainable = False

    inputs = keras.Input(shape=(*IMG_SIZE, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation="relu")(x)   # 🔥 plus large que Mobile/Efficient
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(lr),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model, base_model


In [13]:
model, base_model = build_resnet50(NUM_CLASSES, LR_HEAD)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,650,122 (94.03 MB)

 Trainable params: 1,058,314 (4.04 MB)

 Non-trainable params: 23,591,808 (90.00 MB)

**Entraînement HEAD ONLY**

In [14]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=4,
        restore_best_weights=True
    )
]

history_head = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_HEAD,
    callbacks=callbacks
)

Epoch 1/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 61s 983ms/step - accuracy: 0.6027 - loss: 1.5665 - val_accuracy: 0.6760 - val_loss: 0.9091
Epoch 2/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 56s 962ms/step - accuracy: 0.7207 - loss: 1.0315 - val_accuracy: 0.7339 - val_loss: 0.8799
Epoch 3/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 56s 967ms/step - accuracy: 0.7707 - loss: 0.8373 - val_accuracy: 0.7554 - val_loss: 0.7875
Epoch 4/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 56s 969ms/step - accuracy: 0.7870 - loss: 0.7922 - val_accuracy: 0.8112 - val_loss: 0.5872
Epoch 5/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 56s 962ms/step - accuracy: 0.7880 - loss: 0.7753 - val_accuracy: 0.7897 - val_loss: 0.6957
Epoch 6/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 57s 980ms/step - accuracy: 0.7929 - loss: 0.7587 - val_accuracy: 0.8219 - val_loss: 0.5751
Epoch 7/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 56s 962ms/step - accuracy: 0.8071 - loss: 0.6608 - val_accuracy: 0.8283 - val_loss: 0.6529
Epoch 8/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 56s 959ms/step - accuracy: 0.8060 - loss: 0.6244 - val_accu

**Fine-Tuning partiel**

In [15]:
total_layers = len(base_model.layers)
FINE_TUNE_AT = total_layers - 30

for layer in base_model.layers[:FINE_TUNE_AT]:
    layer.trainable = False

for layer in base_model.layers[FINE_TUNE_AT:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

print("Couches entraînables:",
      sum(l.trainable for l in base_model.layers))


Couches entraînables: 21


In [16]:
model.compile(
    optimizer=keras.optimizers.Adam(LR_FINE),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

**Entraînement Fine-Tuning**

In [17]:
callbacks_fine = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=4,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.3,
        patience=2,
        min_lr=1e-6
    )
]
history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FINE,
    callbacks=callbacks_fine
)

Epoch 1/15
58/58 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.7984 - loss: 0.6341 - val_accuracy: 0.6288 - val_loss: 1.1265 - learning_rate: 1.0000e-04
Epoch 2/15
58/58 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.8353 - loss: 0.4913 - val_accuracy: 0.8069 - val_loss: 0.6190 - learning_rate: 1.0000e-04
Epoch 3/15
58/58 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.8614 - loss: 0.4130 - val_accuracy: 0.8112 - val_loss: 0.5856 - learning_rate: 1.0000e-04
Epoch 4/15
58/58 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.8707 - loss: 0.3845 - val_accuracy: 0.8348 - val_loss: 0.4954 - learning_rate: 1.0000e-04
Epoch 5/15
58/58 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.8679 - loss: 0.3645 - val_accuracy: 0.7575 - val_loss: 0.6751 - learning_rate: 1.0000e-04
Epoch 6/15
58/58 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.8935 - loss: 0.3171 - val_accuracy: 0.7940 - val_loss: 0.7015 - learning_rate: 1.0000e-04
Epoch 7/15
58/58 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.8978 - loss:

**Évaluation finale**

In [18]:
val_loss, val_acc = model.evaluate(val_ds)
print(f"✅ ResNet50 – Val accuracy finale : {val_acc:.4f}")

15/15 ━━━━━━━━━━━━━━━━━━━━ 11s 733ms/step - accuracy: 0.8648 - loss: 0.4483
✅ ResNet50 – Val accuracy finale : 0.8648


**Sauvegarde modèle + métadonnées**

In [19]:
MODEL_PATH = "resnet50_gc10_finetuned.h5"
model.save(MODEL_PATH)

metadata = {
    "model": "ResNet50",
    "img_size": IMG_SIZE,
    "num_classes": NUM_CLASSES,
    "val_accuracy": float(val_acc)
}

with open("resnet50_metadata.json", "w") as f:
    json.dump(metadata, f, indent=4)

print("💾 Modèle ResNet50 sauvegardé")


💾 Modèle ResNet50 sauvegardé
